In [1]:

####################################################################################################
####################################################################################################

import sys
import os.path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

import keras
import keras.utils.vis_utils
from keras import backend as K
from keras import layers
from keras.layers import Dense
from keras.models import load_model
from keras.models import Sequential
from keras.utils import plot_model

sys.path.append(os.path.abspath("../"))
from helpers import *

sys.path.append(os.path.abspath("../../"))
from pax_utils import s1s2_utils

from get_lstm_data import *


Using TensorFlow backend.


In [2]:

####################################################################################################
####################################################################################################

nEventsTrain = 1000

input_dir  = "../../pax_merge/merged/"
input_file = 'merged_all_0k_test.pkl'
input_path = input_dir + input_file

df_events, s2_window_max = getEventsDataFrame(input_path)

df_events = df_events[nEventsTrain:][:].copy(deep=True)

print(df_events.shape)
#display(df_events)

#train_data, train_truth  = get_data(df_events, s2_window_max)

print()
print("Event Max S2 Window size: " + str(s2_window_max))
print()
#print("Event Input data shape: " + str(event_train_data.shape ))
#print("Event Truth data shape: " + str(event_train_truth.shape))
#print("Input data shape:       " + str(train_data.shape ))
#print("Truth data shape:       " + str(train_truth.shape))
print()
        


(250, 155)

Event Max S2 Window size: 1448




In [3]:

####################################################################################################
# Test Data
####################################################################################################

train_data, train_truth  = get_data(df_events, s2_window_max)


print()
#print("Event Input data shape: " + str(event_train_data.shape ))
#print("Event Truth data shape: " + str(event_train_truth.shape))
print("Input data shape:       " + str(train_data.shape ))
print("Truth data shape:       " + str(train_truth.shape))
print()

0 empty events

Input data shape:       (250, 1448, 127)
Truth data shape:       (250, 2)



In [4]:

####################################################################################################
####################################################################################################

model_name_h5     = "models/model_xy_s2waveforms_lstm_elu.h5"



In [5]:

####################################################################################################
# Predict
####################################################################################################

model       = load_model(model_name_h5)
arr_xy_pred = model.predict(train_data)

print("\nLoaded Model: " + model_name_h5)
print("Predicted\n")




Loaded Model: models/model_xy_s2waveforms_lstm_elu.h5
Predicted



In [6]:

####################################################################################################
####################################################################################################

#arr_x_true = df_train_truth[:]['x'].as_matrix()
#arr_y_true = df_train_truth[:]['y'].as_matrix()

arr_x_true = train_truth[:, 0]
arr_y_true = train_truth[:, 1]

arr_x_pred = arr_xy_pred[:, 0]
arr_y_pred = arr_xy_pred[:, 1]

arr_dx     = arr_x_true - arr_x_pred
arr_dy     = arr_y_true - arr_y_pred


print(arr_x_pred.shape)
print(arr_y_pred.shape)
print(arr_x_true.shape)
print(arr_y_true.shape)


(250,)
(250,)
(250,)
(250,)


In [7]:

####################################################################################################
####################################################################################################

file_out         = "./predictions/xy_s2waveforms.pkl"
df_out           = pd.DataFrame()

df_out['x_pred'] = arr_x_pred
df_out['y_pred'] = arr_y_pred

df_out['x_true'] = arr_x_true
df_out['y_true'] = arr_y_true


df_out.to_pickle(file_out)


####################################################################################################
####################################################################################################

df_test = pd.read_pickle(file_out)

display(df_test[:][:])
print()


,x_pred,y_pred,x_true,y_true
0,-0.016128,0.113763,-10.306364,-29.337755
1,-0.016128,0.113763,-5.860157,-38.058211
2,-0.016128,0.113763,2.076336,-6.685439
3,-0.016128,0.113763,13.068567,11.037362
4,-0.016128,0.113763,3.147869,-8.235632
5,-0.016128,0.113763,3.101128,36.751105
6,-0.016128,0.113763,-8.692201,-17.955263
7,-0.016128,0.113763,23.526641,7.091572
8,-0.016128,0.113763,22.036806,-24.966093
9,-0.016128,0.113763,-25.482662,-8.773385
